In [1]:
import pandas as pd
import pathlib
import geopandas as gpd
import matplotlib.pyplot as plt
import pypsa
import numpy as np
import seaborn as sbn
import cartopy.crs as ccrs
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Load the input data

In [78]:
base_path = pathlib.Path("/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/")
base_network_pypsa_earth = pypsa.Network(pathlib.Path(base_path, "geothermal-modelling/workflow/pypsa-earth/networks/US_2021/base.nc"))
base_network_pypsa_usa = pypsa.Network(pathlib.Path(base_path, "pypsa-usa/workflow/resources/Default/usa/elec_base_network.nc"))
eia_base_network = gpd.read_file(pathlib.Path(base_path, "US_Electric_Power_Transmission_Lines_5037807202786552385.geojson"))

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '24035', '24036', '24037', '24038', '24039', '24040', '24041', '24042',
       '24043', '24044'],
      dtype='object', name='name', length=24028)
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '24035', '24036', '24037', '24038', '24039', '24040', '24041', '24042',
       '24043', '24044'],
      dtype='object', name='name', length=24028)
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '24035', '24036', '24037', '24038', '24039', '24040', '24041', '24042',
       '24043', '24044'],
      dtype='object', name='name', length=24028)
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '24035', '24036', '24037', '24038', '24039', '24040', '24041', '24042',
       '24043', '24044'],
      dtype='object', name='name', length=24028)
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '24035', '24036', '24037', '24038', '

In [65]:
eia_base_network.head(5)

,OBJECTID_1,OBJECTID,ID,TYPE,STATUS,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,OWNER,VOLTAGE,VOLT_CLASS,INFERRED,SUB_1,SUB_2,SHAPE__Len,GlobalID,geometry,geometry_type
0,1,1,100511,AC; Overhead,In Service,221121,Electric Bulk Power Transmission and Control,"Imagery, EIA 861, EIA 860","Tue, 08 Nov 2016 00:00:00 GMT",Imagery,"Mon, 27 Mar 2017 00:00:00 GMT",Not Available,115.0,100-161,Yes,Waters River,Tap136228,79.523416,f1500a3a-804a-4da4-b6f6-bd0fbb2cf36d,"LINESTRING (-70.92855 42.54359, -70.92841 42.5...",LineString
1,2,2,140809,AC; Overhead,In Service,221121,Electric Bulk Power Transmission and Control,"Imagery, EIA 860","Wed, 16 Apr 2014 00:00:00 GMT",Imagery,"Wed, 15 Feb 2017 00:00:00 GMT",Kingsport Power Co,161.0,100-161,Yes,Phillips Bend,John Sevier,24018.513114,58c24e54-d758-4071-9ebf-d83beb60c707,"LINESTRING (-82.96123 36.37699, -82.9609 36.37...",LineString
2,3,3,140837,AC; Overhead,In Service,221121,Electric Bulk Power Transmission and Control,"Imagery, https://www9.nationalgridus.com/oasis...","Tue, 16 Jun 2015 00:00:00 GMT",Imagery,"Tue, 05 Mar 2019 00:00:00 GMT",Niagara Mohawk Power Corp.,115.0,100-161,Yes,Tap140359,Tap140373,5972.919614,c40bbff1-b1ed-4905-9ff6-1a80b7b422fc,"LINESTRING (-75.33407 43.08946, -75.33426 43.0...",LineString
3,4,4,140811,AC; Overhead,In Service,221121,Electric Bulk Power Transmission and Control,"Imagery, OpenStreetMap","Fri, 20 Jun 2014 00:00:00 GMT",Imagery,"Mon, 20 Mar 2017 00:00:00 GMT","Entergy New Orleans, Llc",115.0,100-161,Yes,Gentilly Road,Michoud Station,14253.092823,f09483e3-5419-4b94-aac5-a6450f2452c4,"LINESTRING (-89.93728 30.00905, -89.93773 30.0...",LineString
4,5,5,140813,AC; Overhead,In Service,221121,Electric Bulk Power Transmission and Control,"Imagery, EIA 860","Tue, 04 Oct 2016 00:00:00 GMT",Imagery/other,"Wed, 09 May 2018 00:00:00 GMT",Midamerican Energy Co,161.0,100-161,Yes,Unknown137689,Tap137690,8950.844317,36ac4f5e-ad18-47c2-bb7d-8ba64b7fa822,"LINESTRING (-90.56609 41.61256, -90.56468 41.6...",LineString


The total number of lines in the EIA dataset is

In [49]:
eia_base_network["geometry"] = eia_base_network["geometry"].boundary
eia_base_network[["ID", "TYPE", "VOLTAGE", "VOLT_CLASS", "SUB_1", "SUB_2", "SHAPE__Len", "geometry"]].head(2)

,ID,TYPE,VOLTAGE,VOLT_CLASS,SUB_1,SUB_2,SHAPE__Len,geometry
0,100511,AC; Overhead,115.0,100-161,Waters River,Tap136228,79.523416,"MULTIPOINT ((-70.92855 42.54359), (-70.92841 4..."
1,140809,AC; Overhead,161.0,100-161,Phillips Bend,John Sevier,24018.513114,"MULTIPOINT ((-82.96123 36.37699), (-82.80788 3..."


In [55]:
eia_base_network.shape

(94216, 20)

At first glance, it is possible to see that some transmission lines correspond to a voltage of -999999.0 kV. In particular the number of lines corresponding to -999999.0 is

In [ ]:
eia_base_network.loc[eia_base_network["VOLTAGE"]==-999999.0].shape[0]

which acoounts for almost 15 % of the lines in the EIA dataset

In [ ]:
eia_base_network.loc[eia_base_network["VOLTAGE"]==-999999.0].shape[0]/eia_base_network.shape[0]*100

For the moment, such lines are filtered out

In [ ]:
eia_base_network = eia_base_network.loc[eia_base_network["VOLTAGE"]!=-999999.0]
eia_base_network.shape[0]

Get the voltage bins from the reference data

In [ ]:
eia_voltage_bins_flags = eia_base_network["VOLT_CLASS"].unique().tolist()
eia_voltage_bins_flags

In [ ]:
volt_class_min_values = []
volt_class_max_values = []
for bin_name in eia_voltage_bins_flags:
    volt_class_min_values.append(eia_base_network.loc[eia_base_network["VOLT_CLASS"]==bin_name]["VOLTAGE"].min())
    volt_class_max_values.append(eia_base_network.loc[eia_base_network["VOLT_CLASS"]==bin_name]["VOLTAGE"].max())
volt_class_bin_limits = pd.DataFrame(
    {
        "voltage_class": eia_voltage_bins_flags,
        "voltage_class_min": volt_class_min_values,
        "voltage_class_max": volt_class_max_values,
    }
)
volt_class_bin_limits

The voltage classes in the EIA dataset include also *Dc*. The number of lines corresponding to such voltage class is

In [ ]:
eia_base_network.loc[eia_base_network["VOLT_CLASS"]=='Dc'].shape[0]

Considering that the PyPSA-Earth base network does not contain any DC line

In [ ]:
base_network_pypsa_earth.lines["dc"].unique()

the lines corresponding to the *Dc* voltage class are removed from the EIA dataset

In [ ]:
eia_base_network = eia_base_network.loc[eia_base_network["VOLT_CLASS"]!='Dc']
eia_base_network.shape[0]

Furthermore, the lines corresponding to the voltage class *Not Available* are removed as well

In [ ]:
eia_base_network = eia_base_network.loc[eia_base_network["VOLT_CLASS"]!='Not Available']
eia_base_network.shape[0]

In [ ]:
eia_base_network["STATUS"].unique()

In [ ]:
base_network_pypsa_earth.lines.shape[0]

# PyPSA-Earth - EIA comparison

Add the voltage class to the PyPSA-Earth base network

In [80]:
base_network_pypsa_earth.lines["v_nom_class"] = base_network_pypsa_earth.lines["v_nom"]
base_network_pypsa_earth.lines.head(2)

,tag_type,v_nom,num_parallel,tag_frequency,bus0,bus1,length,underground,under_construction,dc,country,geometry,bounds,bus_0_coors,bus_1_coors,bus0_lon,bus0_lat,bus1_lon,bus1_lat,carrier,type,s_max_pu,s_nom,underwater_fraction,x,r,g,b,s_nom_mod,s_nom_extendable,s_nom_min,s_nom_max,capital_cost,build_year,lifetime,terrain_factor,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,line_width,v_nom_class
Line,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,line,138.0,2.0,60.0,3501,5496,NaN,False,False,False,US,"LINESTRING (-98.2396 26.2599, -98.40958 26.160...","MULTIPOINT ((-98.2396 26.2599), (-98.1734 26.1...",POINT (-98.2396 26.2599),POINT (-98.1734 26.1203),-98.2396,26.2599,-98.1734,26.1203,AC,243-AL1/39-ST1A 20.0,0.7,308.339685,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,0.0,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,138.0
1,line,115.0,1.0,60.0,714,10809,NaN,False,False,False,US,"LINESTRING (-106.4285 31.7916, -106.3723559999...","MULTIPOINT ((-106.4285 31.7916), (-106.2873 31...",POINT (-106.4285 31.7916),POINT (-106.2873 31.7802),-106.4285,31.7916,-106.2873,31.7802,AC,243-AL1/39-ST1A 20.0,0.7,128.474869,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,0.0,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,115.0


In [81]:
v_nom_class_dict_pypsa_earth = {
    55.: 'Under 100',
    57.1: 'Under 100', 
    60.: 'Under 100',
    66.: 'Under 100',  
    69.: 'Under 100',  
    70.: 'Under 100',  
    88.: 'Under 100', 
    92.: 'Under 100', 
    100.: "100-161",
    115.: "100-161", 
    120.: "100-161",  
    125.: "100-161",  
    138.: "100-161",  
    160.: "100-161",  
    161.: "100-161",
    220.: "220-287",
    230.: "220-287",
    287.: "220-287",
    345.: "345",
    500.: "500",
    765.: "735 And Above"
}

In [82]:
base_network_pypsa_earth.lines["v_nom_class"] = base_network_pypsa_earth.lines["v_nom_class"].replace(v_nom_class_dict_pypsa_earth)
base_network_pypsa_earth.lines.head(2)

,tag_type,v_nom,num_parallel,tag_frequency,bus0,bus1,length,underground,under_construction,dc,country,geometry,bounds,bus_0_coors,bus_1_coors,bus0_lon,bus0_lat,bus1_lon,bus1_lat,carrier,type,s_max_pu,s_nom,underwater_fraction,x,r,g,b,s_nom_mod,s_nom_extendable,s_nom_min,s_nom_max,capital_cost,build_year,lifetime,terrain_factor,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,line_width,v_nom_class
Line,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,line,138.0,2.0,60.0,3501,5496,NaN,False,False,False,US,"LINESTRING (-98.2396 26.2599, -98.40958 26.160...","MULTIPOINT ((-98.2396 26.2599), (-98.1734 26.1...",POINT (-98.2396 26.2599),POINT (-98.1734 26.1203),-98.2396,26.2599,-98.1734,26.1203,AC,243-AL1/39-ST1A 20.0,0.7,308.339685,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,0.0,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100-161
1,line,115.0,1.0,60.0,714,10809,NaN,False,False,False,US,"LINESTRING (-106.4285 31.7916, -106.3723559999...","MULTIPOINT ((-106.4285 31.7916), (-106.2873 31...",POINT (-106.4285 31.7916),POINT (-106.2873 31.7802),-106.4285,31.7916,-106.2873,31.7802,AC,243-AL1/39-ST1A 20.0,0.7,128.474869,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,0.0,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100-161


In [88]:
base_network_pypsa_earth.lines["line_width"] = 0.0
base_network_pypsa_earth.lines.loc[
        base_network_pypsa_earth.lines["v_nom_class"] == "100-161", "line_width"] = base_network_pypsa_earth.lines.s_nom * 1000/100000
base_network_pypsa_earth.lines.head(2)

,tag_type,v_nom,num_parallel,tag_frequency,bus0,bus1,length,underground,under_construction,dc,country,geometry,bounds,bus_0_coors,bus_1_coors,bus0_lon,bus0_lat,bus1_lon,bus1_lat,carrier,type,s_max_pu,s_nom,underwater_fraction,x,r,g,b,s_nom_mod,s_nom_extendable,s_nom_min,s_nom_max,capital_cost,build_year,lifetime,terrain_factor,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,line_width,v_nom_class
Line,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,line,138.0,2.0,60.0,3501,5496,NaN,False,False,False,US,"LINESTRING (-98.2396 26.2599, -98.40958 26.160...","MULTIPOINT ((-98.2396 26.2599), (-98.1734 26.1...",POINT (-98.2396 26.2599),POINT (-98.1734 26.1203),-98.2396,26.2599,-98.1734,26.1203,AC,243-AL1/39-ST1A 20.0,0.7,308.339685,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,0.0,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.083397,100-161
1,line,115.0,1.0,60.0,714,10809,NaN,False,False,False,US,"LINESTRING (-106.4285 31.7916, -106.3723559999...","MULTIPOINT ((-106.4285 31.7916), (-106.2873 31...",POINT (-106.4285 31.7916),POINT (-106.2873 31.7802),-106.4285,31.7916,-106.2873,31.7802,AC,243-AL1/39-ST1A 20.0,0.7,128.474869,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,0.0,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.284749,100-161


### Count plot

In [ ]:
sbn.countplot(data=base_network_pypsa_earth.lines, x=base_network_pypsa_earth.lines["v_nom_class"]).set(title='PyPSA-Earth base network')

In [ ]:
sbn.countplot(data=eia_base_network, x=eia_base_network["VOLT_CLASS"], color="orange").set(title='EIA network topology')

### Maps

In [ ]:
# Available voltage classes are:
# Under 100
# 100-161
# 220-287
# 345
# 500
# 735 And Above

selected_voltage_class_earth = "735 And Above"

In [ ]:
base_network_pe_volt_class = base_network_pypsa_earth.lines.loc[base_network_pypsa_earth.lines["v_nom_class"] == selected_voltage_class_earth]
base_network_pe_volt_class = gpd.GeoDataFrame(base_network_pe_volt_class, geometry=gpd.GeoSeries.from_wkt(base_network_pe_volt_class.geometry), crs="EPSG:4326")
base_network_pe_volt_class["union_geo"] = 0
base_network_pe_volt_class = base_network_pe_volt_class.dissolve(by="union_geo")
base_network_pe_volt_class = base_network_pe_volt_class.to_crs(3857)

eia_base_network_volt_class = eia_base_network.loc[eia_base_network["VOLT_CLASS"]==selected_voltage_class_earth]
eia_base_network_volt_class["union_geo"] = 0
eia_base_network_volt_class = eia_base_network_volt_class.dissolve(by="union_geo")
eia_base_network_volt_class = eia_base_network_volt_class.to_crs(3857)

In [ ]:
base_network_pe_volt_class.geometry

In [ ]:
intersection_geometries = base_network_pe_volt_class.intersection(eia_base_network_volt_class)
fig, ax = plt.subplots(figsize=(20, 4))
ax.set_axis_off()

intersection_geometries.plot(ax=ax, color="black", markersize=5)
base_network_pe_volt_class.plot(ax=ax, color="blue", kind="geo")
eia_base_network_volt_class.plot(ax=ax, color="orange", kind="geo")

In [ ]:
!pip install -U kaleido

In [ ]:
import plotly.express as px
import shapely
import kaleido

lats = []
lons = []

for feature in zip(base_network_pe_volt_class.geometry):
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)

fig = px.line_map(lat=lats, lon=lons)
fig.write_image("plotly_test.png")

In [ ]:
fig.savefig("network_comparison_for_voltage_class_{}.png".format(str(selected_voltage_class_earth)))

In [ ]:
eia_base_network_765["union_geo"] = 0
eia_base_network_new = eia_base_network_765.dissolve(by="union_geo")
eia_base_network_new

In [ ]:
base_network_pypsa_earth.lines["line_width"] = 0.0
base_network_pypsa_earth.lines.loc[base_network_pypsa_earth.lines["v_nom_class"] == selected_voltage_class_earth, "line_width"] = 1.0

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, subplot_kw={"projection":ccrs.PlateCarree(), "projection":ccrs.PlateCarree()}, figsize=(20, 4))
base_network_pypsa_earth.plot(line_widths = base_network_pypsa_earth.lines["line_width"], bus_sizes = 0, line_colors = "blue", ax=ax1)
base_network_pypsa_earth.plot(line_widths = base_network_pypsa_earth.lines["line_width"], bus_sizes = 0, line_colors = "white", ax=ax2)
eia_base_network.loc[eia_base_network["VOLT_CLASS"]==selected_voltage_class_earth].plot(ax=ax2, color="orange")
fig.suptitle("Comparison for voltage class: {}".format(selected_voltage_class_earth))
ax1.set_aspect('equal')
ax1.title.set_text("PyPSA-Earth base network")
ax2.set_aspect("equal")
ax2.title.set_text("EIA network")

# PyPSA-USA - EIA comparison

Add the voltage class to the PyPSA-USA base network

In [ ]:
base_network_pypsa_usa.lines["v_nom_class"] = base_network_pypsa_usa.lines["v_nom"]
base_network_pypsa_usa.lines.head(2)

In [ ]:
base_network_pypsa_usa.lines.v_nom.unique()

In [ ]:
v_nom_class_dict_pypsa_usa = {
    69.: 'Under 100',
    100.: "100-161",
    138.: "100-161", 
    115.: "100-161", 
    161.: "100-161",
    230.: "220-287",
    345.: "345",
    500.: "500",
    765.: "735 And Above"
}

In [ ]:
base_network_pypsa_usa.lines["v_nom_class"] = base_network_pypsa_usa.lines["v_nom_class"].replace(v_nom_class_dict_pypsa_usa)

### Maps

In [ ]:
# Available voltage classes are:
# Under 100
# 100-161
# 220-287
# 345
# 500
# 735 And Above

selected_voltage_class_usa = "735 And Above"

In [ ]:
base_network_pypsa_usa.lines["line_width"] = 0.0
base_network_pypsa_usa.lines.loc[base_network_pypsa_usa.lines["v_nom_class"] == selected_voltage_class_usa, "line_width"] = 1.0

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharey=True, subplot_kw={"projection":ccrs.PlateCarree(), "projection":ccrs.PlateCarree()}, figsize=(20, 4))
base_network_pypsa_usa.plot(line_widths = base_network_pypsa_usa.lines["line_width"], bus_sizes = 0, line_colors = "blue", ax=ax1)
base_network_pypsa_usa.plot(line_widths = base_network_pypsa_usa.lines["line_width"], bus_sizes = 0, line_colors = "white", ax=ax2)
eia_base_network.loc[eia_base_network["VOLT_CLASS"]==selected_voltage_class_usa].plot(ax=ax2, color="orange")
fig.suptitle("Comparison for voltage class: {}".format(selected_voltage_class_usa))
ax1.set_aspect('equal')
ax1.title.set_text("PyPSA-USA base network")
ax2.set_aspect("equal")
ax2.title.set_text("EIA network")

In [ ]:
fig.savefig("network_comparison_pusa_for_voltage_class_{}.png".format(str(selected_voltage_class_usa)))

## Compare crossings

In [50]:
gadm_usa_json_path_level_1 = "/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/geothermal-modelling/analysis/data/gadm41_USA_1.json"
gadm_gdp_usa_level_1 = gpd.read_file(gadm_usa_json_path_level_1)

In [51]:
gadm_gdp_usa_level_1.head(2)

,GID_1,GID_0,COUNTRY,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,ISO_1,geometry
0,USA.1_1,USA,UnitedStates,Alabama,AL|Ala.,NA,State,State,NA,US.AL,US-AL,"MULTIPOLYGON (((-88.12060 30.23320, -88.12330 ..."
1,USA.2_1,USA,UnitedStates,Alaska,AK|Alaska,NA,State,State,NA,US.AK,US-AK,"MULTIPOLYGON (((-179.13150 51.20970, -179.1328..."


In [52]:
gadm_gdp_usa_level_1.shape

(51, 12)

Apply a spatial join between the reference transmission lines topology from EIA and the GADM shapes (level 1)

In [53]:
spatial_join_gadm_eia = eia_base_network.sjoin(gadm_gdp_usa_level_1, how="inner")
spatial_join_gadm_eia.head(2)

,OBJECTID_1,OBJECTID,ID,TYPE,STATUS,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,OWNER,VOLTAGE,VOLT_CLASS,INFERRED,SUB_1,SUB_2,SHAPE__Len,GlobalID,geometry,index_right,GID_1,GID_0,COUNTRY,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,ISO_1
0,1,1,100511,AC; Overhead,In Service,221121,Electric Bulk Power Transmission and Control,"Imagery, EIA 861, EIA 860","Tue, 08 Nov 2016 00:00:00 GMT",Imagery,"Mon, 27 Mar 2017 00:00:00 GMT",Not Available,115.0,100-161,Yes,Waters River,Tap136228,79.523416,f1500a3a-804a-4da4-b6f6-bd0fbb2cf36d,"MULTIPOINT ((-70.92855 42.54359), (-70.92841 4...",21,USA.22_1,USA,UnitedStates,Massachusetts,CommonwealthofMassachusetts|MA,NA,State,State,NA,US.MA,US-MA
1,2,2,140809,AC; Overhead,In Service,221121,Electric Bulk Power Transmission and Control,"Imagery, EIA 860","Wed, 16 Apr 2014 00:00:00 GMT",Imagery,"Wed, 15 Feb 2017 00:00:00 GMT",Kingsport Power Co,161.0,100-161,Yes,Phillips Bend,John Sevier,24018.513114,58c24e54-d758-4071-9ebf-d83beb60c707,"MULTIPOINT ((-82.96123 36.37699), (-82.80788 3...",42,USA.43_1,USA,UnitedStates,Tennessee,TN|Tenn.,NA,State,State,NA,US.TN,US-TN


In [54]:
spatial_join_gadm_eia.shape

(95483, 32)

In [56]:
spatial_join_gadm_eia = spatial_join_gadm_eia[["ID", "TYPE", "GID_1", "VOLTAGE", "VOLT_CLASS", "SUB_1", "SUB_2", "ISO_1", "NAME_1", "SHAPE__Len", "geometry"]]

In [57]:
spatial_join_gadm_eia.head(2)

,ID,TYPE,GID_1,VOLTAGE,VOLT_CLASS,SUB_1,SUB_2,ISO_1,NAME_1,SHAPE__Len,geometry
0,100511,AC; Overhead,USA.22_1,115.0,100-161,Waters River,Tap136228,US-MA,Massachusetts,79.523416,"MULTIPOINT ((-70.92855 42.54359), (-70.92841 4..."
1,140809,AC; Overhead,USA.43_1,161.0,100-161,Phillips Bend,John Sevier,US-TN,Tennessee,24018.513114,"MULTIPOINT ((-82.96123 36.37699), (-82.80788 3..."


In [58]:
spatial_join_gadm_eia.shape

(95483, 11)

In [62]:
count_df = (spatial_join_gadm_eia.groupby(["ID"])["ID"].count()-1).to_frame(name="crossings_count").reset_index()
count_df.head(2)

,ID,crossings_count
0,100000,0
1,100001,0


In [63]:
spatial_join_gadm_eia = pd.merge(spatial_join_gadm_eia, count_df, how="inner", on="ID")
spatial_join_gadm_eia.shape

(95483, 13)

In [64]:
spatial_join_gadm_eia.head(2)

,ID,TYPE,GID_1,VOLTAGE,VOLT_CLASS,SUB_1,SUB_2,ISO_1,NAME_1,SHAPE__Len,geometry,crossings_count_x,crossings_count_y
0,100511,AC; Overhead,USA.22_1,115.0,100-161,Waters River,Tap136228,US-MA,Massachusetts,79.523416,"MULTIPOINT ((-70.92855 42.54359), (-70.92841 4...",1,0
1,140809,AC; Overhead,USA.43_1,161.0,100-161,Phillips Bend,John Sevier,US-TN,Tennessee,24018.513114,"MULTIPOINT ((-82.96123 36.37699), (-82.80788 3...",1,0


In [17]:
spatial_join_gadm_eia["SHAPE__Len"].max()

1781469.77660848

In [18]:
spatial_join_gadm_eia[spatial_join_gadm_eia["SHAPE__Len"]==1781469.77660848]

,ID,TYPE,GID_1,VOLTAGE,VOLT_CLASS,SUB_1,SUB_2,ISO_1,NAME_1,SHAPE__Len,geometry
69088,200823,DC,USA.5_1,1000.0,Dc,Celilo,Sylmar East,US-CA,California,1.781470e+06,"MULTIPOINT ((-121.11308 45.59461), (-118.48113..."
69088,200823,DC,USA.38_1,1000.0,Dc,Celilo,Sylmar East,US-OR,Oregon,1.781470e+06,"MULTIPOINT ((-121.11308 45.59461), (-118.48113..."


The line above is the longest and it goes from around LA to Oregon.

Apply a spatial join between the transmission lines topology from the PyPSA-Earth base network and the GADM shapes (level 1)

In [ ]:
base_network_pypsa_earth_geopandas = gpd.GeoDataFrame(base_network_pypsa_earth.lines, geometry=gpd.GeoSeries.from_wkt(base_network_pypsa_earth.lines.geometry), crs="EPSG:4326")

In [ ]:
spatial_join_gadm_pypsa_earth = base_network_pypsa_earth_geopandas.sjoin(gadm_gdp_usa_level_1, how="inner")
spatial_join_gadm_pypsa_earth.head(5)

In [ ]:
spatial_join_gadm_pypsa_earth[["GID_1", "v_nom", "v_nom_class", "type", "num_parallel", "ISO_1", "NAME_1"]].head(2)

In [35]:
eia_base_network.tail(2)

,OBJECTID_1,OBJECTID,ID,TYPE,STATUS,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,OWNER,VOLTAGE,VOLT_CLASS,INFERRED,SUB_1,SUB_2,SHAPE__Len,GlobalID,geometry,boundaries,sub_1_coors
94214,94215,94215,202496,AC; Overhead,Not Available,221121,Electric Bulk Power Transmission and Control,Imagery,"Wed, 10 Apr 2019 00:00:00 GMT",Imagery,"Wed, 10 Apr 2019 00:00:00 GMT",City Of Gering - (Ne),115.0,100-161,No,Tap161754,Tap204258,49097.232454,17d49594-e00f-4a93-b602-fe5992196552,"LINESTRING (-103.87027 41.81278, -103.87065 41...","MULTIPOINT ((-103.87027 41.81278), (-104.26522...",None
94215,94216,94216,166911,AC; Overhead,Not Available,221121,Electric Bulk Power Transmission and Control,Imagery,"Wed, 10 Apr 2019 00:00:00 GMT",Imagery,"Wed, 10 Apr 2019 00:00:00 GMT",City Of Gering - (Ne),115.0,100-161,No,Unknown202234,Tap161753,24810.070426,5c9be9e1-1546-49d3-9a3c-f1db7bc776ca,"LINESTRING (-103.66255 41.83121, -103.66285 41...","MULTIPOINT ((-103.66255 41.83121), (-103.86845...",None


In [58]:
eia_base_network["geometry_type"] = eia_base_network.geom_type
len(eia_base_network[eia_base_network["geometry_type"]=="MultiLineString"]["OBJECTID_1"].values)

110

In [51]:
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==2791] # --> line to remove (220-287)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==13822] # --> line to remove (Under 100)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==76120] # --> line to remove (100-161)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==91845] # --> line to remove (Under 100)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==92557] # --> line to remove (Under 100)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==93203] # --> line to remove (Under 100)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==93215] # --> line to remove (Under 100)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==93327] # --> line to remove (Under 100)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==93408] # --> line to remove (Under 100)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==93603] # --> line to remove (Under 100)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==93649] # --> line to remove (Under 100)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==93687] # --> line to remove (Not Available)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==93852] # --> line to remove (220-287)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==93986] # --> line to remove (Under 100)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==93852] # --> line to remove (220-287)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==93987] # --> line to remove (Under 100)
#eia_base_network_copy = eia_base_network[eia_base_network["OBJECTID_1"]==94063] # --> line to remove (Under 100)
eia_base_network_copy = eia_base_network[(eia_base_network["OBJECTID_1"]>13822) & (eia_base_network["OBJECTID_1"]<23000)]
eia_base_network_copy["boundaries"] = eia_base_network_copy["geometry"].boundary
eia_base_network_copy[["boundaries_0", "boundaries_1"]]= eia_base_network_copy["boundaries"].explode(index_parts=True).unstack()
#eia_base_network_copy["boundaries"].explode(index_parts=True).unstack().tail()
#eia_base_network_copy.tail(10)

/opt/anaconda3/envs/pypsa-earth-most-advanced/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/envs/pypsa-earth-most-advanced/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/anaconda3/envs/pypsa-earth-most-advanced/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is

In [37]:
eia_base_network[eia_base_network["OBJECTID_1"]==13822]

,OBJECTID_1,OBJECTID,ID,TYPE,STATUS,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,OWNER,VOLTAGE,VOLT_CLASS,INFERRED,SUB_1,SUB_2,SHAPE__Len,GlobalID,geometry
13821,13822,13822,126003,AC; Overhead,In Service,221121,Electric Bulk Power Transmission and Control,"Imagery, OpenStreetMap, EIA 861, EIA 860","Tue, 07 Oct 2014 00:00:00 GMT",Imagery,"Mon, 10 Dec 2018 00:00:00 GMT",Oklahoma Gas & Electric Co,69.0,Under 100,Yes,Unknown118590,Tap159847,13707.512421,ecd1eea4-f438-40c7-8850-65b5f7fb4f1a,"MULTILINESTRING ((-97.12892 35.13081, -97.1291..."


In [8]:
df = eia_base_network[eia_base_network["OBJECTID_1"]==94063]["geometry"].line_merge(directed=False).line_merge(directed=False).line_merge(directed=False)
df

94062    MULTILINESTRING ((-117.25164 34.14995, -117.25...
dtype: geometry

In [72]:
base_network_pypsa_earth.lines.head(2)

,tag_type,v_nom,num_parallel,tag_frequency,bus0,bus1,length,underground,under_construction,dc,country,geometry,bounds,bus_0_coors,bus_1_coors,bus0_lon,bus0_lat,bus1_lon,bus1_lat,carrier,type,s_max_pu,s_nom,underwater_fraction,x,r,g,b,s_nom_mod,s_nom_extendable,s_nom_min,s_nom_max,capital_cost,build_year,lifetime,terrain_factor,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,v_nom_class
Line,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,line,138.0,2.0,60.0,3501,5496,NaN,False,False,False,US,"LINESTRING (-98.2396 26.2599, -98.40958 26.160...","MULTIPOINT ((-98.2396 26.2599), (-98.1734 26.1...",POINT (-98.2396 26.2599),POINT (-98.1734 26.1203),-98.2396,26.2599,-98.1734,26.1203,AC,243-AL1/39-ST1A 20.0,0.7,308.339685,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,0.0,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100-161
1,line,115.0,1.0,60.0,714,10809,NaN,False,False,False,US,"LINESTRING (-106.4285 31.7916, -106.3723559999...","MULTIPOINT ((-106.4285 31.7916), (-106.2873 31...",POINT (-106.4285 31.7916),POINT (-106.2873 31.7802),-106.4285,31.7916,-106.2873,31.7802,AC,243-AL1/39-ST1A 20.0,0.7,128.474869,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,inf,0.0,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100-161
